In [1]:
import os
import glob
import pandas as pd

In [2]:
# Define working directory
os.chdir("C:/Users/samya/VS Code Projects/Python/Python Projects/Spotify_hit_predictor/backend")

In [3]:
ds00 = pd.read_csv('Original_data/dataset-of-00s.csv')
ds10 = pd.read_csv('Original_data/dataset-of-10s.csv')
ds60 = pd.read_csv('Original_data/dataset-of-60s.csv')
ds70 = pd.read_csv('Original_data/dataset-of-70s.csv')
ds80 = pd.read_csv('Original_data/dataset-of-80s.csv')
ds90 = pd.read_csv('Original_data/dataset-of-90s.csv')

merge = [ds00, ds10, ds60, ds70, ds80, ds90]
df = pd.concat(merge)
df.reset_index(drop=True, inplace=True)

In [4]:
df2 = df.drop(['track', 'artist', 'uri'], axis=1)

In [5]:
#Validation of correlation between variables
import numpy as np 
df3=df2.corr()

In [6]:
# Sin key & tempo
data = df2
X = data[['danceability', 'energy', 'loudness','mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness','valence', 'duration_ms', 'time_signature','chorus_hit', 'sections']]  
y = data.target

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

## Data Preprocessing

In [8]:
#Scale features (X) using MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler(feature_range=(0,1)).fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test) 

X_train_scaled

array([[0.78947368, 0.52283395, 0.76172236, ..., 0.8       , 0.19905999,
        0.06508876],
       [0.54048583, 0.83094117, 0.77921392, ..., 0.8       , 0.21633373,
        0.04142012],
       [0.5       , 0.66588373, 0.7455139 , ..., 0.8       , 0.20480589,
        0.03550296],
       ...,
       [0.57692308, 0.36777999, 0.68303866, ..., 0.8       , 0.15844718,
        0.09467456],
       [0.41902834, 0.16070793, 0.50819858, ..., 0.8       , 0.200772  ,
        0.04733728],
       [0.29554656, 0.24873856, 0.67937808, ..., 0.6       , 0.12021911,
        0.06508876]])

One-hot encode the labels

In [9]:
#One-hot encode output labels (y)
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

y_train_categorical

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

## Creating and defining our Deep Learning Model Architecture

In [10]:
#Create a sequential model
from tensorflow.keras.models import Sequential
model = Sequential() 

from tensorflow.keras.layers import Dense
number_inputs = 13  

#Create hidden layers
model.add(Dense(units=14,activation='relu', input_dim=number_inputs))
model.add(Dense(units=120,activation='relu'))
model.add(Dense(units=80,activation='relu'))

#Create output layer
number_classes = 2
model.add(Dense(units=number_classes, activation='softmax')) 

Number of Hidden Nodes 
https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw#:~:text=The%20number%20of%20hidden%20neurons,size%20of%20the%20input%20layer.

In [11]:
#Model Summary
model.summary() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 14)                196       
                                                                 
 dense_1 (Dense)             (None, 120)               1800      
                                                                 
 dense_2 (Dense)             (None, 80)                9680      
                                                                 
 dense_3 (Dense)             (None, 2)                 162       
                                                                 
Total params: 11,838
Trainable params: 11,838
Non-trainable params: 0
_________________________________________________________________


In [12]:
#Compile the Model
import tensorflow as tf
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

## Training the Model

In [13]:
#Training the Model
history = model.fit(X_train_scaled, y_train_categorical, epochs=500, batch_size=2000, shuffle=True, verbose=2)

Epoch 1/500
16/16 - 0s - loss: 0.6579 - accuracy: 0.6254 - 393ms/epoch - 25ms/step
Epoch 2/500
16/16 - 0s - loss: 0.5918 - accuracy: 0.6829 - 62ms/epoch - 4ms/step
Epoch 3/500
16/16 - 0s - loss: 0.5614 - accuracy: 0.6984 - 48ms/epoch - 3ms/step
Epoch 4/500
16/16 - 0s - loss: 0.5463 - accuracy: 0.7111 - 50ms/epoch - 3ms/step
Epoch 5/500
16/16 - 0s - loss: 0.5378 - accuracy: 0.7195 - 45ms/epoch - 3ms/step
Epoch 6/500
16/16 - 0s - loss: 0.5298 - accuracy: 0.7280 - 48ms/epoch - 3ms/step
Epoch 7/500
16/16 - 0s - loss: 0.5223 - accuracy: 0.7344 - 43ms/epoch - 3ms/step
Epoch 8/500
16/16 - 0s - loss: 0.5162 - accuracy: 0.7421 - 51ms/epoch - 3ms/step
Epoch 9/500
16/16 - 0s - loss: 0.5111 - accuracy: 0.7455 - 56ms/epoch - 3ms/step
Epoch 10/500
16/16 - 0s - loss: 0.5066 - accuracy: 0.7486 - 44ms/epoch - 3ms/step
Epoch 11/500
16/16 - 0s - loss: 0.5044 - accuracy: 0.7488 - 52ms/epoch - 3ms/step
Epoch 12/500
16/16 - 0s - loss: 0.5021 - accuracy: 0.7516 - 30ms/epoch - 2ms/step
Epoch 13/500
16/16 - 0s

Epoch 101/500
16/16 - 0s - loss: 0.4688 - accuracy: 0.7752 - 50ms/epoch - 3ms/step
Epoch 102/500
16/16 - 0s - loss: 0.4674 - accuracy: 0.7766 - 39ms/epoch - 2ms/step
Epoch 103/500
16/16 - 0s - loss: 0.4673 - accuracy: 0.7764 - 34ms/epoch - 2ms/step
Epoch 104/500
16/16 - 0s - loss: 0.4673 - accuracy: 0.7774 - 41ms/epoch - 3ms/step
Epoch 105/500
16/16 - 0s - loss: 0.4681 - accuracy: 0.7768 - 51ms/epoch - 3ms/step
Epoch 106/500
16/16 - 0s - loss: 0.4675 - accuracy: 0.7771 - 42ms/epoch - 3ms/step
Epoch 107/500
16/16 - 0s - loss: 0.4668 - accuracy: 0.7766 - 42ms/epoch - 3ms/step
Epoch 108/500
16/16 - 0s - loss: 0.4658 - accuracy: 0.7768 - 32ms/epoch - 2ms/step
Epoch 109/500
16/16 - 0s - loss: 0.4657 - accuracy: 0.7775 - 43ms/epoch - 3ms/step
Epoch 110/500
16/16 - 0s - loss: 0.4657 - accuracy: 0.7772 - 42ms/epoch - 3ms/step
Epoch 111/500
16/16 - 0s - loss: 0.4651 - accuracy: 0.7787 - 31ms/epoch - 2ms/step
Epoch 112/500
16/16 - 0s - loss: 0.4653 - accuracy: 0.7761 - 42ms/epoch - 3ms/step
Epoc

Epoch 200/500
16/16 - 0s - loss: 0.4546 - accuracy: 0.7843 - 31ms/epoch - 2ms/step
Epoch 201/500
16/16 - 0s - loss: 0.4541 - accuracy: 0.7840 - 35ms/epoch - 2ms/step
Epoch 202/500
16/16 - 0s - loss: 0.4548 - accuracy: 0.7829 - 24ms/epoch - 1ms/step
Epoch 203/500
16/16 - 0s - loss: 0.4551 - accuracy: 0.7831 - 47ms/epoch - 3ms/step
Epoch 204/500
16/16 - 0s - loss: 0.4530 - accuracy: 0.7845 - 26ms/epoch - 2ms/step
Epoch 205/500
16/16 - 0s - loss: 0.4533 - accuracy: 0.7852 - 37ms/epoch - 2ms/step
Epoch 206/500
16/16 - 0s - loss: 0.4531 - accuracy: 0.7844 - 31ms/epoch - 2ms/step
Epoch 207/500
16/16 - 0s - loss: 0.4537 - accuracy: 0.7845 - 31ms/epoch - 2ms/step
Epoch 208/500
16/16 - 0s - loss: 0.4535 - accuracy: 0.7854 - 50ms/epoch - 3ms/step
Epoch 209/500
16/16 - 0s - loss: 0.4543 - accuracy: 0.7843 - 36ms/epoch - 2ms/step
Epoch 210/500
16/16 - 0s - loss: 0.4562 - accuracy: 0.7833 - 31ms/epoch - 2ms/step
Epoch 211/500
16/16 - 0s - loss: 0.4535 - accuracy: 0.7836 - 31ms/epoch - 2ms/step
Epoc

Epoch 299/500
16/16 - 0s - loss: 0.4463 - accuracy: 0.7884 - 47ms/epoch - 3ms/step
Epoch 300/500
16/16 - 0s - loss: 0.4464 - accuracy: 0.7881 - 31ms/epoch - 2ms/step
Epoch 301/500
16/16 - 0s - loss: 0.4472 - accuracy: 0.7893 - 36ms/epoch - 2ms/step
Epoch 302/500
16/16 - 0s - loss: 0.4455 - accuracy: 0.7889 - 44ms/epoch - 3ms/step
Epoch 303/500
16/16 - 0s - loss: 0.4454 - accuracy: 0.7892 - 44ms/epoch - 3ms/step
Epoch 304/500
16/16 - 0s - loss: 0.4452 - accuracy: 0.7894 - 47ms/epoch - 3ms/step
Epoch 305/500
16/16 - 0s - loss: 0.4462 - accuracy: 0.7883 - 38ms/epoch - 2ms/step
Epoch 306/500
16/16 - 0s - loss: 0.4459 - accuracy: 0.7891 - 47ms/epoch - 3ms/step
Epoch 307/500
16/16 - 0s - loss: 0.4462 - accuracy: 0.7881 - 47ms/epoch - 3ms/step
Epoch 308/500
16/16 - 0s - loss: 0.4467 - accuracy: 0.7881 - 41ms/epoch - 3ms/step
Epoch 309/500
16/16 - 0s - loss: 0.4457 - accuracy: 0.7885 - 47ms/epoch - 3ms/step
Epoch 310/500
16/16 - 0s - loss: 0.4462 - accuracy: 0.7896 - 49ms/epoch - 3ms/step
Epoc

Epoch 398/500
16/16 - 0s - loss: 0.4419 - accuracy: 0.7912 - 47ms/epoch - 3ms/step
Epoch 399/500
16/16 - 0s - loss: 0.4411 - accuracy: 0.7919 - 47ms/epoch - 3ms/step
Epoch 400/500
16/16 - 0s - loss: 0.4388 - accuracy: 0.7934 - 37ms/epoch - 2ms/step
Epoch 401/500
16/16 - 0s - loss: 0.4382 - accuracy: 0.7926 - 44ms/epoch - 3ms/step
Epoch 402/500
16/16 - 0s - loss: 0.4384 - accuracy: 0.7942 - 44ms/epoch - 3ms/step
Epoch 403/500
16/16 - 0s - loss: 0.4389 - accuracy: 0.7923 - 42ms/epoch - 3ms/step
Epoch 404/500
16/16 - 0s - loss: 0.4385 - accuracy: 0.7930 - 44ms/epoch - 3ms/step
Epoch 405/500
16/16 - 0s - loss: 0.4400 - accuracy: 0.7917 - 39ms/epoch - 2ms/step
Epoch 406/500
16/16 - 0s - loss: 0.4391 - accuracy: 0.7930 - 41ms/epoch - 3ms/step
Epoch 407/500
16/16 - 0s - loss: 0.4389 - accuracy: 0.7928 - 41ms/epoch - 3ms/step
Epoch 408/500
16/16 - 0s - loss: 0.4386 - accuracy: 0.7923 - 47ms/epoch - 3ms/step
Epoch 409/500
16/16 - 0s - loss: 0.4391 - accuracy: 0.7927 - 40ms/epoch - 3ms/step
Epoc

Epoch 497/500
16/16 - 0s - loss: 0.4322 - accuracy: 0.7959 - 43ms/epoch - 3ms/step
Epoch 498/500
16/16 - 0s - loss: 0.4320 - accuracy: 0.7957 - 41ms/epoch - 3ms/step
Epoch 499/500
16/16 - 0s - loss: 0.4353 - accuracy: 0.7938 - 35ms/epoch - 2ms/step
Epoch 500/500
16/16 - 0s - loss: 0.4327 - accuracy: 0.7958 - 49ms/epoch - 3ms/step


## Validation of the Model

In [14]:
#Compare Model performace between training and testing data
model_loss_train, model_accuracy_train = model.evaluate(X_train_scaled, y_train_categorical, verbose=2)
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)

print(f"Train --> Loss: {model_loss_train}, Accuracy: {model_accuracy_train}")    
print(f"Test --> Loss: {model_loss}, Accuracy: {model_accuracy}")

964/964 - 1s - loss: 0.4330 - accuracy: 0.7948 - 807ms/epoch - 838us/step
322/322 - 0s - loss: 0.4721 - accuracy: 0.7776 - 226ms/epoch - 702us/step
Train --> Loss: 0.4330178499221802, Accuracy: 0.794771134853363
Test --> Loss: 0.47205522656440735, Accuracy: 0.7775615453720093


## Saving the Trained Model

In [15]:
# Save the model
model.save("../Models/h5/skey&tempo.h5")